## 📦 Setup

In [25]:
import sys
import os
from pathlib import Path

# Add project root
project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root))

print(f"📁 Project: {project_root}")

📁 Project: /home/sakana/Code/RAG-bidding


In [26]:
import psycopg2
import json
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv
from typing import Dict, List, Any
import warnings
warnings.filterwarnings('ignore')

# Load environment
load_dotenv()

# Database config
DB_CONFIG = {
    'host': 'localhost',
    'database': 'rag_bidding_v2',
    'user': 'sakana',
    'password': 'sakana123'
}

print("✅ Imports successful")

✅ Imports successful


In [27]:
# Helper functions

def get_connection():
    """Get database connection."""
    return psycopg2.connect(**DB_CONFIG)

def run_query(query: str, params: tuple = None) -> pd.DataFrame:
    """Run query and return DataFrame."""
    conn = get_connection()
    try:
        df = pd.read_sql_query(query, conn, params=params)
        return df
    finally:
        conn.close()

def run_query_dict(query: str, params: tuple = None) -> List[Dict]:
    """Run query and return list of dicts."""
    conn = get_connection()
    try:
        cursor = conn.cursor()
        cursor.execute(query, params)
        columns = [desc[0] for desc in cursor.description]
        results = [dict(zip(columns, row)) for row in cursor.fetchall()]
        return results
    finally:
        conn.close()

def print_section(title: str):
    """Print formatted section header."""
    print("\n" + "="*80)
    print(f"📊 {title}")
    print("="*80 + "\n")

print("✅ Helper functions loaded")

✅ Helper functions loaded


---

## 🔍 Part 1: Database Overview

### 1.1: List All Tables

In [28]:
print_section("Database Tables")

query = """
SELECT 
    schemaname,
    tablename,
    pg_size_pretty(pg_total_relation_size(schemaname||'.'||tablename)) as size
FROM pg_tables
WHERE schemaname NOT IN ('pg_catalog', 'information_schema')
ORDER BY pg_total_relation_size(schemaname||'.'||tablename) DESC;
"""

tables_df = run_query(query)
print(tables_df.to_string(index=False))

print(f"\n📋 Total tables: {len(tables_df)}")


📊 Database Tables

schemaname               tablename   size
    public  langchain_pg_embedding 112 MB
    public               documents 176 kB
    public langchain_pg_collection  48 kB

📋 Total tables: 3


### 1.2: Vector DB Table Structure (langchain_pg_embedding)

In [29]:
print_section("Vector DB Table Structure")

# Get column information
query = """
SELECT 
    column_name,
    data_type,
    character_maximum_length,
    is_nullable
FROM information_schema.columns
WHERE table_name = 'langchain_pg_embedding'
ORDER BY ordinal_position;
"""

columns_df = run_query(query)
print("📋 Columns:")
print(columns_df.to_string(index=False))

# Get row count
count_query = "SELECT COUNT(*) as total_chunks FROM langchain_pg_embedding;"
count_df = run_query(count_query)

print(f"\n📊 Total chunks: {count_df['total_chunks'].iloc[0]:,}")


📊 Vector DB Table Structure

📋 Columns:
  column_name         data_type character_maximum_length is_nullable
           id character varying                     None          NO
collection_id              uuid                     None         YES
    embedding      USER-DEFINED                     None         YES
     document character varying                     None         YES
    cmetadata             jsonb                     None         YES

📊 Total chunks: 6,242


### 1.3: Documents Table Structure

In [30]:
print_section("Documents Table Structure")

# Get column information
query = """
SELECT 
    column_name,
    data_type,
    character_maximum_length,
    is_nullable,
    column_default
FROM information_schema.columns
WHERE table_name = 'documents'
ORDER BY ordinal_position;
"""

try:
    columns_df = run_query(query)
    print("📋 Columns:")
    print(columns_df.to_string(index=False))
    
    # Get row count
    count_query = "SELECT COUNT(*) as total_documents FROM documents;"
    count_df = run_query(count_query)
    
    print(f"\n📊 Total documents: {count_df['total_documents'].iloc[0]}")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Documents table may not exist yet")


📊 Documents Table Structure

📋 Columns:
  column_name                   data_type  character_maximum_length is_nullable              column_default
           id                        uuid                       NaN          NO           gen_random_uuid()
  document_id           character varying                     255.0          NO                        None
document_name                        text                       NaN          NO                        None
     category           character varying                     100.0          NO                        None
document_type           character varying                      50.0          NO                        None
  source_file                        text                       NaN          NO                        None
    file_name                        text                       NaN          NO                        None
 total_chunks                     integer                       NaN         YES                

---

## 🔬 Part 2: Sample Data Analysis

### 2.1: Sample Chunks from Vector DB

In [31]:
print_section("Sample Chunks - Vector DB")

query = """
SELECT 
    id as chunk_uuid,
    LEFT(document, 100) as content_preview,
    cmetadata->>'document_id' as document_id,
    cmetadata->>'chunk_id' as chunk_id,
    cmetadata->>'chunk_index' as chunk_index,
    cmetadata->>'document_type' as doc_type,
    cmetadata->>'title' as title,
    cmetadata->>'source_file' as source_file
FROM langchain_pg_embedding
ORDER BY cmetadata->>'document_id', (cmetadata->>'chunk_index')::int
LIMIT 10;
"""

samples_df = run_query(query)

print("📄 First 10 chunks:\n")
for idx, row in samples_df.iterrows():
    print(f"[{idx+1}] document_id: {row['document_id']}")
    print(f"    chunk_id: {row['chunk_id']}")
    print(f"    chunk_index: {row['chunk_index']}")
    print(f"    type: {row['doc_type']}")
    print(f"    title: {row['title'][:60]}..." if row['title'] and len(str(row['title'])) > 60 else f"    title: {row['title']}")
    print(f"    source: {Path(row['source_file']).name if row['source_file'] else 'N/A'}")
    print(f"    content: {row['content_preview']}...")
    print()


📊 Sample Chunks - Vector DB

📄 First 10 chunks:

[1] document_id: bidding_untitled
    chunk_id: bidding_untitled_form_0000
    chunk_index: 0
    type: bidding
    title: None
    source: N/A
    content: [Section: Mẫu số 01A. Mẫu Tờ trình kế hoạch tổng thể lựa chọn nhà thầu]

Mẫu số 01A. Mẫu Tờ trình kế...

[2] document_id: bidding_untitled
    chunk_id: bidding_untitled_form_0000
    chunk_index: 0
    type: bidding
    title: None
    source: N/A
    content: [Section: Mẫu số 01 (Webform trên Hệ thống)]

Mẫu số 01 (Webform trên Hệ thống)
BẢNG KÊ HẠNG MỤC CÔN...

[3] document_id: bidding_untitled
    chunk_id: bidding_untitled_form_0000
    chunk_index: 0
    type: bidding
    title: None
    source: N/A
    content: [Section: Mẫu số 4.5. Mẫu Báo cáo tình hình thực hiện Kết luận kiểm tra]

Mẫu số 4.5. Mẫu Báo cáo tì...

[4] document_id: bidding_untitled
    chunk_id: bidding_untitled_form_0000
    chunk_index: 0
    type: bidding
    title: None
    source: N/A
    content: [Sectio

### 2.2: Analyze document_id Format Distribution

In [32]:
print_section("Document ID Format Distribution")

query = """
WITH doc_formats AS (
    SELECT 
        cmetadata->>'document_id' as document_id,
        CASE 
            WHEN cmetadata->>'document_id' ~ '^LUA-' THEN 'Luật (LUA-)'
            WHEN cmetadata->>'document_id' ~ '^ND-' THEN 'Nghị định (ND-)'
            WHEN cmetadata->>'document_id' ~ '^TT-' THEN 'Thông tư (TT-)'
            WHEN cmetadata->>'document_id' ~ '^QD-' THEN 'Quyết định (QD-)'
            WHEN cmetadata->>'document_id' ~ '^FORM-' THEN 'Biểu mẫu (FORM-)'
            WHEN cmetadata->>'document_id' ~ '^TEMPLATE-' THEN 'Mẫu (TEMPLATE-)'
            WHEN cmetadata->>'document_id' ~ '^EXAM-' THEN 'Câu hỏi thi (EXAM-)'
            WHEN cmetadata->>'document_id' LIKE '%untitled%' THEN '⚠️ Old Format (untitled)'
            ELSE 'Other'
        END as format_type
    FROM langchain_pg_embedding
)
SELECT 
    format_type,
    COUNT(DISTINCT document_id) as unique_documents,
    COUNT(*) as total_chunks,
    ROUND(AVG(CASE WHEN document_id IS NOT NULL THEN 1 ELSE 0 END) * 100, 2) as pct
FROM doc_formats
GROUP BY format_type
ORDER BY total_chunks DESC;
"""

format_df = run_query(query)
print(format_df.to_string(index=False))

# Check for old format
old_format = format_df[format_df['format_type'].str.contains('untitled', case=False)]
if not old_format.empty:
    print(f"\n⚠️ WARNING: Found {old_format['total_chunks'].sum()} chunks with OLD format!")
else:
    print("\n✅ All chunks use NEW format (no 'untitled' found)")


📊 Document ID Format Distribution



             format_type  unique_documents  total_chunks   pct
        Biểu mẫu (FORM-)                38          2876 100.0
             Luật (LUA-)                 4          1154 100.0
⚠️ Old Format (untitled)                 1           767 100.0
         Mẫu (TEMPLATE-)                10           722 100.0
         Nghị định (ND-)                 1           595 100.0
          Thông tư (TT-)                 2           123 100.0
        Quyết định (QD-)                 1             5 100.0

⚠️ WARNING: Found 767 chunks with OLD format!


### 2.3: Sample Metadata Fields

In [33]:
print_section("Sample Metadata Fields Analysis")

# Get one full metadata example
query = """
SELECT cmetadata
FROM langchain_pg_embedding
WHERE cmetadata->>'document_id' ~ '^LUA-'
LIMIT 1;
"""

result = run_query_dict(query)

if result:
    metadata = result[0]['cmetadata']
    
    print("📋 Example Metadata (LUA document):\n")
    print(json.dumps(metadata, indent=2, ensure_ascii=False))
    
    print("\n🔑 Available Keys:")
    for i, key in enumerate(sorted(metadata.keys()), 1):
        value_type = type(metadata[key]).__name__
        value_preview = str(metadata[key])[:50]
        print(f"  {i:2d}. {key:30s} ({value_type:10s}): {value_preview}..." if len(str(metadata[key])) > 50 else f"  {i:2d}. {key:30s} ({value_type:10s}): {value_preview}")


📊 Sample Metadata Fields Analysis

📋 Example Metadata (LUA document):

{
  "level": "dieu",
  "chunk_id": "LUA-57-2024-QH15_dieu_0143",
  "has_list": true,
  "has_table": false,
  "hierarchy": [
    "Điều 5. Điều khoản thi hành"
  ],
  "char_count": 618,
  "chunk_index": 118,
  "document_id": "LUA-57-2024-QH15",
  "source_file": "data/raw/Luat chinh/Luat so 57 2024 QH15.docx",
  "total_chunks": 128,
  "document_info": {
    "document_status": "active"
  },
  "document_type": "law",
  "section_title": "Điều khoản thi hành",
  "extra_metadata": {
    "muc": null,
    "phan": null,
    "chuong": null,
    "batch_name": null,
    "dieu_number": "5",
    "khoan_number": null,
    "pipeline_version": "working_upload_pipeline_v1.0",
    "processing_time_ms": 35
  },
  "parent_context": null,
  "is_complete_unit": true,
  "processing_metadata": {
    "retry_count": 0,
    "error_message": null,
    "last_processed_at": "2025-11-09T14:31:57.585012",
    "processing_status": "completed"
  }
}



### 2.4: Metadata Fields Availability

In [34]:
print_section("Metadata Fields Coverage")

# Check common metadata fields
fields_to_check = [
    'document_id',
    'chunk_id',
    'chunk_index',
    'document_type',
    'title',
    'source_file',
    'dieu',
    'khoan',
    'diem',
    'hierarchy',
    'published_date',
    'effective_date',
    'processing_metadata',
    'document_info'
]

coverage_data = []

for field in fields_to_check:
    query = f"""
    SELECT 
        COUNT(*) as total,
        COUNT(CASE WHEN cmetadata->'{field}' IS NOT NULL THEN 1 END) as has_field,
        ROUND(COUNT(CASE WHEN cmetadata->'{field}' IS NOT NULL THEN 1 END)::numeric / COUNT(*) * 100, 2) as coverage_pct
    FROM langchain_pg_embedding;
    """
    
    result = run_query(query)
    coverage_data.append({
        'field': field,
        'has_field': result['has_field'].iloc[0],
        'total': result['total'].iloc[0],
        'coverage_pct': result['coverage_pct'].iloc[0]
    })

coverage_df = pd.DataFrame(coverage_data)
coverage_df = coverage_df.sort_values('coverage_pct', ascending=False)

print(coverage_df.to_string(index=False))

print("\n📊 Summary:")
print(f"   ✅ 100% coverage: {len(coverage_df[coverage_df['coverage_pct'] == 100.0])} fields")
print(f"   ⚠️  < 100% coverage: {len(coverage_df[coverage_df['coverage_pct'] < 100.0])} fields")


📊 Metadata Fields Coverage

              field  has_field  total  coverage_pct
        document_id       6242   6242        100.00
           chunk_id       6242   6242        100.00
        chunk_index       6242   6242        100.00
      document_type       6242   6242        100.00
          hierarchy       6242   6242        100.00
        source_file       6239   6242         99.95
processing_metadata       4708   6242         75.42
      document_info       4708   6242         75.42
              title          0   6242          0.00
               dieu          0   6242          0.00
               diem          0   6242          0.00
              khoan          0   6242          0.00
     effective_date          0   6242          0.00
     published_date          0   6242          0.00

📊 Summary:
   ✅ 100% coverage: 5 fields
   ⚠️  < 100% coverage: 9 fields
              field  has_field  total  coverage_pct
        document_id       6242   6242        100.00
           ch

---

## 📊 Part 3: Documents Table Analysis

### 3.1: Documents Table Sample Data

In [35]:
print_section("Documents Table - Sample Data")

query = """
SELECT 
    document_id,
    document_name,
    document_type,
    category,
    total_chunks,
    status,
    published_date,
    effective_date,
    created_at
FROM documents
ORDER BY created_at DESC
LIMIT 10;
"""

try:
    docs_df = run_query(query)
    
    print("📄 Sample documents:\n")
    for idx, row in docs_df.iterrows():
        print(f"[{idx+1}] {row['document_id']}")
        print(f"    Name: {row['document_name']}")
        print(f"    Type: {row['document_type']} | Category: {row['category']}")
        print(f"    Chunks: {row['total_chunks']} | Status: {row['status']}")
        print(f"    Published: {row['published_date']} | Effective: {row['effective_date']}")
        print()
        
except Exception as e:
    print(f"❌ Error: {e}")


📊 Documents Table - Sample Data

❌ Error: Execution failed on sql '
SELECT 
    document_id,
    document_name,
    document_type,
    category,
    total_chunks,
    status,
    published_date,
    effective_date,
    created_at
FROM documents
ORDER BY created_at DESC
LIMIT 10;
': column "published_date" does not exist
LINE 9:     published_date,
            ^



### 3.2: Documents vs Chunks Consistency Check

In [36]:
print_section("Documents ↔ Chunks Consistency Check")

# Compare documents table with vector DB
query = """
WITH vector_db_docs AS (
    SELECT 
        cmetadata->>'document_id' as document_id,
        COUNT(*) as chunks_in_vector_db
    FROM langchain_pg_embedding
    GROUP BY cmetadata->>'document_id'
),
documents_table AS (
    SELECT 
        document_id,
        total_chunks as chunks_in_documents_table
    FROM documents
)
SELECT 
    COALESCE(v.document_id, d.document_id) as document_id,
    COALESCE(v.chunks_in_vector_db, 0) as vector_db,
    COALESCE(d.chunks_in_documents_table, 0) as documents_table,
    CASE 
        WHEN v.chunks_in_vector_db = d.chunks_in_documents_table THEN '✅ Match'
        WHEN v.chunks_in_vector_db IS NULL THEN '❌ Only in documents table'
        WHEN d.chunks_in_documents_table IS NULL THEN '❌ Only in vector DB'
        ELSE '⚠️ Mismatch'
    END as status
FROM vector_db_docs v
FULL OUTER JOIN documents_table d ON v.document_id = d.document_id
ORDER BY 
    CASE 
        WHEN v.chunks_in_vector_db = d.chunks_in_documents_table THEN 1
        ELSE 0
    END,
    document_id
LIMIT 20;
"""

try:
    consistency_df = run_query(query)
    print(consistency_df.to_string(index=False))
    
    # Summary
    match_count = len(consistency_df[consistency_df['status'] == '✅ Match'])
    mismatch_count = len(consistency_df[consistency_df['status'].str.contains('Mismatch|Only')])
    
    print(f"\n📊 Summary:")
    print(f"   ✅ Consistent: {match_count}")
    print(f"   ⚠️  Inconsistent: {mismatch_count}")
    
except Exception as e:
    print(f"❌ Error: {e}")


📊 Documents ↔ Chunks Consistency Check

              document_id  vector_db  documents_table                    status
EXAM-Ngân-hàng-câu-hỏi-CC          0                0 ❌ Only in documents table
EXAM-Ngân-hàng-câu-hỏi-th          0                0 ❌ Only in documents table
  EXAM-NHCH_2692025_dot-2          0                0 ❌ Only in documents table
  EXAM-NHCH_30925_bo_sung          0                0 ❌ Only in documents table
         bidding_untitled        767              767                   ✅ Match
          FORM-01-Phụ-lục         48               48                   ✅ Match
  FORM-041A-Mẫu-Kế-hoạch-          1                1                   ✅ Match
  FORM-041B-Mẫu-Kế-hoạch-         10               10                   ✅ Match
  FORM-042-Mẫu-Đề-cương-b         10               10                   ✅ Match
  FORM-043-Mẫu-Báo-cáo-ki          8                8                   ✅ Match
  FORM-044-Mẫu-Kết-luận-k          4                4                   ✅ Match

### 3.3: Documents by Type and Category

In [37]:
print_section("Documents Distribution by Type & Category")

query = """
SELECT 
    document_type,
    category,
    COUNT(*) as doc_count,
    SUM(total_chunks) as total_chunks,
    ROUND(AVG(total_chunks), 2) as avg_chunks_per_doc
FROM documents
GROUP BY document_type, category
ORDER BY doc_count DESC;
"""

try:
    dist_df = run_query(query)
    print(dist_df.to_string(index=False))
    
    print(f"\n📊 Total: {dist_df['doc_count'].sum()} documents, {dist_df['total_chunks'].sum():,} chunks")
    
except Exception as e:
    print(f"❌ Error: {e}")


📊 Documents Distribution by Type & Category

  document_type       category  doc_count  total_chunks  avg_chunks_per_doc
   bidding_form Hồ sơ mời thầu         37          2873               77.65
report_template    Mẫu báo cáo         10           722               72.20
  exam_question    Câu hỏi thi          4             0                0.00
            law     Luật chính          4          1154              288.50
       circular       Thông tư          2           123               61.50
        bidding Hồ sơ mời thầu          2           770              385.00
         decree      Nghị định          1           595              595.00
       decision     Quyết định          1             5                5.00

📊 Total: 61 documents, 6,242 chunks


---

## 🔍 Part 4: Old vs New Format Comparison

### 4.1: Check for Old Format Remnants

In [38]:
print_section("Old Format Check")

# Search for old format patterns
query = """
SELECT 
    cmetadata->>'document_id' as document_id,
    cmetadata->>'chunk_id' as chunk_id,
    COUNT(*) as chunk_count
FROM langchain_pg_embedding
WHERE 
    cmetadata->>'document_id' LIKE '%untitled%'
    OR cmetadata->>'chunk_id' LIKE '%untitled%'
GROUP BY cmetadata->>'document_id', cmetadata->>'chunk_id'
ORDER BY chunk_count DESC
LIMIT 10;
"""

old_format_df = run_query(query)

if old_format_df.empty:
    print("✅ No old format (untitled) found in vector DB")
    print("   Migration was successful!")
else:
    print(f"⚠️ Found {len(old_format_df)} chunks with old format:\n")
    print(old_format_df.to_string(index=False))
    
    total_old = old_format_df['chunk_count'].sum()
    print(f"\n❌ Total chunks with old format: {total_old}")


📊 Old Format Check

⚠️ Found 10 chunks with old format:

     document_id                      chunk_id  chunk_count
bidding_untitled    bidding_untitled_form_0000           22
bidding_untitled bidding_untitled_section_0006           18
bidding_untitled bidding_untitled_section_0004           16
bidding_untitled bidding_untitled_section_0007           16
bidding_untitled bidding_untitled_section_0003           16
bidding_untitled    bidding_untitled_form_0002           15
bidding_untitled bidding_untitled_section_0005           15
bidding_untitled    bidding_untitled_form_0001           15
bidding_untitled bidding_untitled_section_0002           15
bidding_untitled bidding_untitled_section_0001           15

❌ Total chunks with old format: 163


### 4.2: New Format Examples

In [39]:
print_section("New Format Examples")

# Get examples of each new format type
format_patterns = {
    'Luật (LUA-)': '^LUA-',
    'Nghị định (ND-)': '^ND-',
    'Thông tư (TT-)': '^TT-',
    'Quyết định (QD-)': '^QD-',
    'Biểu mẫu (FORM-)': '^FORM-',
    'Mẫu (TEMPLATE-)': '^TEMPLATE-',
    'Câu hỏi thi (EXAM-)': '^EXAM-'
}

for format_name, pattern in format_patterns.items():
    query = f"""
    SELECT 
        cmetadata->>'document_id' as document_id,
        cmetadata->>'chunk_id' as chunk_id,
        cmetadata->>'title' as title,
        LEFT(document, 80) as content_preview
    FROM langchain_pg_embedding
    WHERE cmetadata->>'document_id' ~ '{pattern}'
    LIMIT 1;
    """
    
    result = run_query(query)
    
    if not result.empty:
        row = result.iloc[0]
        print(f"\n📄 {format_name}")
        print(f"   document_id: {row['document_id']}")
        print(f"   chunk_id: {row['chunk_id']}")
        print(f"   title: {row['title'][:60]}..." if row['title'] and len(str(row['title'])) > 60 else f"   title: {row['title']}")
        print(f"   content: {row['content_preview']}...")
    else:
        print(f"\n⚠️ {format_name}: No examples found")


📊 New Format Examples


📄 Luật (LUA-)
   document_id: LUA-57-2024-QH15
   chunk_id: LUA-57-2024-QH15_dieu_0143
   title: None
   content: [Section: Điều 5. Điều khoản thi hành]

Điều 5. Điều khoản thi hành
1. Bãi bỏ Đi...

📄 Nghị định (ND-)
   document_id: ND-214-4.8.-CP
   chunk_id: ND-214-4.8.-CP_dieu_0000
   title: None
   content: [Section: Điều 1. Phạm vi điều chỉnh]

Điều 1. Phạm vi điều chỉnh
1. Nghị định n...

📄 Thông tư (TT-)
   document_id: TT-00-Quyết-định-Thông-tư
   chunk_id: TT-00-Quyết-định-Thông-tư_dieu_0000
   title: None
   content: [Section: Điều 1. Phạm vi điều chỉnh]

Điều 1. Phạm vi điều chỉnh
Thông tư này h...

📄 Quyết định (QD-)
   document_id: QD-1667-BYT
   chunk_id: QD-1667-BYT_dieu_0000
   title: None
   content: [Section: Điều 1. Quyết định này quy định việc áp dụng hình thức lựa chọn nhà th...

📄 Biểu mẫu (FORM-)
   document_id: FORM-05-Mẫu-Báo-cáo-đấu-
   chunk_id: FORM-05-Mẫu-Báo-cáo-đấu-_section_0000
   title: None
   content: [Section: I. THÔNG TIN CƠ

---

## 📋 Part 5: Preprocessing Requirements Analysis

### 5.1: What Preprocessing Needs to Do

In [40]:
print_section("Preprocessing Requirements")

print("""
📋 REQUIREMENTS FOR UPLOAD PIPELINE:

1️⃣ **Generate document_id**
   - Format: LUA-*, ND-*, TT-*, QD-*, FORM-*, TEMPLATE-*, EXAM-*
   - Based on document type and filename
   - Must be unique and descriptive

2️⃣ **Extract Document-Level Metadata**
   - document_name: From filename or content
   - document_type: law, decree, circular, decision, bidding, template, exam
   - category: Classify document purpose
   - published_date: Extract from document (if available)
   - effective_date: Extract from document (if available)
   - source_file: Full path to original file

3️⃣ **Insert into documents Table**
   - Create new row for each uploaded document
   - Set status = 'active' (default)
   - Set total_chunks = 0 initially
   - Record created_at, updated_at timestamps

4️⃣ **Generate Chunks with Metadata**
   - chunk_id: {document_id}_{type}_{index}
   - chunk_index: Sequential number
   - Embed chunk content → vector
   - Store in langchain_pg_embedding with full cmetadata

5️⃣ **Update documents.total_chunks**
   - After all chunks inserted
   - Count chunks from vector DB
   - Update documents table

6️⃣ **Consistency Check**
   - Verify all chunks have document_id in documents table
   - Verify total_chunks matches vector DB count
   - Rollback if inconsistent
""")

print("\n" + "="*80)
print("📊 Current State Check")
print("="*80 + "\n")

# Check if preprocessing follows these requirements
query = """
SELECT 
    COUNT(DISTINCT cmetadata->>'document_id') as unique_docs_in_vector_db,
    (SELECT COUNT(*) FROM documents) as docs_in_documents_table
FROM langchain_pg_embedding;
"""

try:
    check_df = run_query(query)
    
    vector_docs = check_df['unique_docs_in_vector_db'].iloc[0]
    table_docs = check_df['docs_in_documents_table'].iloc[0]
    
    print(f"📊 Document Count:")
    print(f"   Vector DB: {vector_docs} unique documents")
    print(f"   Documents table: {table_docs} documents")
    
    if vector_docs == table_docs:
        print(f"\n✅ GOOD: Both tables have same document count")
    elif table_docs < vector_docs:
        print(f"\n⚠️ WARNING: Documents table missing {vector_docs - table_docs} documents")
        print(f"   Preprocessing may not be creating documents table entries")
    else:
        print(f"\n⚠️ WARNING: Documents table has {table_docs - vector_docs} extra documents")
        print(f"   Some documents may not have chunks")
        
except Exception as e:
    print(f"❌ Error: {e}")


📊 Preprocessing Requirements


📋 REQUIREMENTS FOR UPLOAD PIPELINE:

1️⃣ **Generate document_id**
   - Format: LUA-*, ND-*, TT-*, QD-*, FORM-*, TEMPLATE-*, EXAM-*
   - Based on document type and filename
   - Must be unique and descriptive

2️⃣ **Extract Document-Level Metadata**
   - document_name: From filename or content
   - document_type: law, decree, circular, decision, bidding, template, exam
   - category: Classify document purpose
   - published_date: Extract from document (if available)
   - effective_date: Extract from document (if available)
   - source_file: Full path to original file

3️⃣ **Insert into documents Table**
   - Create new row for each uploaded document
   - Set status = 'active' (default)
   - Set total_chunks = 0 initially
   - Record created_at, updated_at timestamps

4️⃣ **Generate Chunks with Metadata**
   - chunk_id: {document_id}_{type}_{index}
   - chunk_index: Sequential number
   - Embed chunk content → vector
   - Store in langchain_pg_embedding wi

### 5.2: Missing Documents Analysis

In [41]:
print_section("Missing Documents Analysis")

# Find documents in vector DB but not in documents table
query = """
WITH vector_docs AS (
    SELECT DISTINCT cmetadata->>'document_id' as document_id
    FROM langchain_pg_embedding
),
table_docs AS (
    SELECT document_id
    FROM documents
)
SELECT v.document_id
FROM vector_docs v
LEFT JOIN table_docs t ON v.document_id = t.document_id
WHERE t.document_id IS NULL
ORDER BY v.document_id;
"""

try:
    missing_df = run_query(query)
    
    if missing_df.empty:
        print("✅ All documents in vector DB exist in documents table")
    else:
        print(f"⚠️ Found {len(missing_df)} documents in vector DB but NOT in documents table:\n")
        for idx, doc_id in enumerate(missing_df['document_id'], 1):
            print(f"  {idx:2d}. {doc_id}")
        
        print("\n💡 Action Required:")
        print("   - These documents were added before documents table was created")
        print("   - Need to backfill documents table with these entries")
        print("   - Or: Update preprocessing to create documents table entries for new uploads")
        
except Exception as e:
    print(f"❌ Error: {e}")


📊 Missing Documents Analysis

✅ All documents in vector DB exist in documents table
✅ All documents in vector DB exist in documents table


---

## 📝 Summary Report

In [42]:
print("\n" + "="*80)
print("📊 DATABASE STRUCTURE SUMMARY")
print("="*80)

# Gather all stats
try:
    # Vector DB stats
    vector_stats = run_query("""
        SELECT 
            COUNT(*) as total_chunks,
            COUNT(DISTINCT cmetadata->>'document_id') as unique_documents,
            COUNT(CASE WHEN cmetadata->>'document_id' ~ '^(LUA|ND|TT|QD|FORM|TEMPLATE|EXAM)-' THEN 1 END) as new_format_chunks,
            COUNT(CASE WHEN cmetadata->>'document_id' LIKE '%untitled%' THEN 1 END) as old_format_chunks
        FROM langchain_pg_embedding;
    """)
    
    # Documents table stats
    docs_stats = run_query("""
        SELECT 
            COUNT(*) as total_documents,
            SUM(total_chunks) as sum_total_chunks,
            COUNT(CASE WHEN total_chunks = 0 THEN 1 END) as docs_with_zero_chunks
        FROM documents;
    """)
    
    vs = vector_stats.iloc[0]
    ds = docs_stats.iloc[0]
    
    print(f"""
🗄️ VECTOR DB (langchain_pg_embedding):
   - Total chunks: {vs['total_chunks']:,}
   - Unique documents: {vs['unique_documents']}
   - New format chunks: {vs['new_format_chunks']:,} ({vs['new_format_chunks']/vs['total_chunks']*100:.1f}%)
   - Old format chunks: {vs['old_format_chunks']:,} ({vs['old_format_chunks']/vs['total_chunks']*100:.1f}%)

📋 DOCUMENTS TABLE:
   - Total documents: {ds['total_documents']}
   - Sum of total_chunks: {ds['sum_total_chunks']}
   - Documents with 0 chunks: {ds['docs_with_zero_chunks']}

🔍 CONSISTENCY:
   - Vector DB unique docs: {vs['unique_documents']}
   - Documents table rows: {ds['total_documents']}
   - Match: {'✅ YES' if vs['unique_documents'] == ds['total_documents'] else '⚠️ NO'}
    """)
    
    # Recommendations
    print("\n" + "="*80)
    print("💡 RECOMMENDATIONS FOR PREPROCESSING UPDATE")
    print("="*80 + "\n")
    
    if vs['old_format_chunks'] > 0:
        print(f"1. ⚠️ Still have {vs['old_format_chunks']} chunks with old format")
        print("   → Need to investigate and re-migrate if necessary\n")
    else:
        print("1. ✅ All chunks use new format\n")
    
    if vs['unique_documents'] != ds['total_documents']:
        print(f"2. ⚠️ Inconsistency: {vs['unique_documents']} docs in vector DB vs {ds['total_documents']} in documents table")
        print("   → Preprocessing must insert into BOTH tables\n")
    else:
        print("2. ✅ Document count consistent\n")
    
    if ds['docs_with_zero_chunks'] > 0:
        print(f"3. ⚠️ {ds['docs_with_zero_chunks']} documents have 0 chunks")
        print("   → These are likely exam PDFs that were never preprocessed")
        print("   → Preprocessing should handle these or mark as 'pending'\n")
    
    print("""
📋 NEXT STEPS:

1. Analyze src/preprocessing/upload_pipeline.py
   - Check if it creates documents table entries
   - Verify document_id generation logic
   - Ensure total_chunks is updated after insertion

2. Test upload endpoint with sample file
   - Upload a new document
   - Verify entry created in documents table
   - Verify chunks inserted into vector DB
   - Verify document_id format is correct

3. Update preprocessing if needed
   - Add DocumentIDGenerator
   - Add documents table insertion
   - Add consistency checks
    """)
    
except Exception as e:
    print(f"❌ Error generating summary: {e}")


📊 DATABASE STRUCTURE SUMMARY

🗄️ VECTOR DB (langchain_pg_embedding):
   - Total chunks: 6,242
   - Unique documents: 57
   - New format chunks: 5,475 (87.7%)
   - Old format chunks: 767 (12.3%)

📋 DOCUMENTS TABLE:
   - Total documents: 61
   - Sum of total_chunks: 6242
   - Documents with 0 chunks: 4

🔍 CONSISTENCY:
   - Vector DB unique docs: 57
   - Documents table rows: 61
   - Match: ⚠️ NO
    

💡 RECOMMENDATIONS FOR PREPROCESSING UPDATE

1. ⚠️ Still have 767 chunks with old format
   → Need to investigate and re-migrate if necessary

2. ⚠️ Inconsistency: 57 docs in vector DB vs 61 in documents table
   → Preprocessing must insert into BOTH tables

3. ⚠️ 4 documents have 0 chunks
   → These are likely exam PDFs that were never preprocessed
   → Preprocessing should handle these or mark as 'pending'


📋 NEXT STEPS:

1. Analyze src/preprocessing/upload_pipeline.py
   - Check if it creates documents table entries
   - Verify document_id generation logic
   - Ensure total_chunks is up

---

## 📝 Notes

**Key Findings:**
- Vector DB structure và data after migration
- Documents table structure và consistency với vector DB
- Old vs new format distribution
- Metadata fields availability

**Next Notebook:**
- Analyze preprocessing pipeline (`src/preprocessing/`)
- Identify what needs to be updated
- Test upload endpoint

**Related Files:**
- Migration notebook: `notebooks/migration/document-structure-migration.ipynb`
- Update plan: `documents/migration/POST_MIGRATION_UPDATE_PLAN.md`